
# LangGraph with Databricks LLM - Code Explanation

This code demonstrates a simple conversational AI workflow using **LangGraph** (a graph-based framework for building language model applications) integrated with **Databricks' LLM serving endpoint**.


## 🤖 **Core Function: `call_llm`**

```python
def call_llm(state: MessagesState):
    system_msg = {"role": "system", "content": "Reply only with plain text. No formatting."}
    all_msgs = [system_msg] + state["messages"]
    return {"messages": [llm.invoke(all_msgs)]}
```

**Purpose**: This function processes the conversation state and generates LLM responses.

**How it works**:
1. **System Message**: Adds a system prompt instructing the LLM to respond in plain text only (no markdown/formatting)
2. **Message Preparation**: Combines the system message with existing conversation messages
3. **LLM Invocation**: Calls the Databricks LLM with the complete message history
4. **State Update**: Returns the new message to be added to the conversation state

## 🕸️ **Graph Construction**

```python
builder = StateGraph(MessagesState)  # Create graph with message state management
builder.add_node("call_llm", call_llm)  # Add the LLM processing node
builder.add_edge(START, "call_llm")  # Connect start to LLM node
builder.add_edge("call_llm", END)  # Connect LLM node to end
graph = builder.compile()  # Compile the graph for execution
```

**Graph Flow**: `START → call_llm → END`

This creates a simple linear workflow where:
- The conversation starts
- Messages are processed by the LLM
- The conversation ends

## 🚀 **Execution**

```python
messages = graph.invoke({"messages": [HumanMessage("Tell me more about Databricks ?")]})
```

**What happens**:
1. **Input**: Creates a human message asking about Databricks
2. **Processing**: The graph processes this through the `call_llm` node
3. **Output**: Returns the complete conversation including the LLM's response
4. **Result**: The `messages` variable contains both the input question and the generated answer

## 🎯 **Key Concepts**

- **StateGraph**: Manages conversation state and message flow
- **MessagesState**: Built-in state type for handling conversation messages
- **Node**: A processing unit in the graph (here, the LLM call)
- **Edges**: Define the flow between nodes
- **Invoke**: Executes the graph with given input

## 💡 **Use Cases**

This pattern is useful for:
- Building conversational AI applications
- Creating chatbots with Databricks LLMs
- Implementing structured conversation flows
- Integrating with larger AI workflows

The code represents a foundational building block that can be extended with additional nodes for more complex behaviors like tool calling, memory management, or multi-step reasoning.

In [0]:
from dotenv import load_dotenv
import os
import random
from typing import Literal, TypedDict
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated
from databricks_langchain import ChatDatabricks
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import MessagesState

# load_dotenv()


# import mlflow
# mlflow.langchain.autolog()





llm = ChatDatabricks(endpoint = "databricks-gpt-oss-120b")

def call_llm(state: MessagesState):

    system_msg = {"role": "system", "content": "Reply only with plain text. No formatting."}
    all_msgs = [system_msg] + state["messages"]

    return {"messages": [llm.invoke(all_msgs)]}


builder = StateGraph(MessagesState)
builder.add_node("call_llm", call_llm)


builder.add_edge(START, "call_llm")
builder.add_edge("call_llm", END)

graph = builder.compile()

messages = graph.invoke({"messages": [HumanMessage("Tell me more about Databricks ?")]})

In [0]:
message = messages['messages'][-1]

for part in message.content:
    if part.get("type") == "text":
       ai_message = part.get("text", "")

print(ai_message)